# Install and Import Dependencies
> You should be able to just click run for this section

In [1]:
pip install geemap geopandas pydeck seaborn geetools mss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (s

In [2]:
# Import packages
import ee
import geemap
import geopandas as gpd
from geetools import batch
import geetools
import os
import mss
import pandas as pd
import numpy as np
import requests
import urllib.parse
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn
import pydeck as pdk
from datetime import datetime
from datetime import timedelta
from sklearn.metrics import r2_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Authenticate and Initialize Google Earth Engine API

In [3]:
# Initialize and Authenticate Earth Engine API
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=KJ6YE9Sk_jusQgPIMGV5Deg6UFQtER2EYqx3hDwMJHI&tc=9tk53DWPbUDWdPpjYdJm43SsPa63MZDndYVdPiN6aPc&cc=wwykprNLeDmhnXH0JgodN9OuoXM8Y5EB3ZY3DQ0yr9w

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7iBNdNplgJgHsS7mWwNQfIR6HzEUGmKRof0O4ehPrKMJA2zSwBquc

Successfully saved authorization token.


# Adjust the slider to pick the Study Period

In [4]:
# These widgets will be used to customize the subsequent call of functions:

# Pick the range of years for study 

year_range = widgets.IntRangeSlider(
    value=[2000, 2022],
    min=2000,
    max=2023,
    step=1,
    description='Years',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

year_range

IntRangeSlider(value=(2000, 2022), description='Years', max=2023, min=2000)

# Snow Change Detection Map

> Includes the following functions:


# *   **studyAreaImporter(path)**
  Imports the .shp file of interest

# *   **snowCoverData()**
  Runs through the input range of years and reduces the studyArea's Terra MODIS Daily 'NDSI_Snow_Cover' product into median values for each date and collects it all into a single pandas DataFrame.

---

In [5]:
def studyAreaImporter(path):
  # Example path: "/content/drive/MyDrive/Term/Working Code/Omeed/Shapefiles/study_area/study_area.shp"

  # Convert shapefile to ee.Geometry() via geemap
  Study_Area = geemap.shp_to_ee(path)
  # Return Study Area(s)
  return Study_Area


def snowCoverData(imageCollection,band):
  # This function will be used to access the NDSI_Snow_Cover band of Terra MODIS
  # and create a pandas DataFrame for each region of interest.



  # The initial and final years are stored below and are also used for the 
  # loading bar.
  
  # Loading bar widget parameters

  i_year = year_range.value[0]
  f_year = year_range.value[1]

  Reducer_Progress = widgets.IntProgress(
      value=0,
      min=0,
      max=f_year-i_year,
      step=1,
      description='Loading:',
      bar_style='', # 'success', 'info', 'warning', 'danger' or ''
      orientation='horizontal')
  
  display(Reducer_Progress)

  # Reduce the data to median and variance

  # Reduce_median will be used to reduce the regions spatially into their
  # Median/Variance values. This will take some time
  # Scale, a parameter of .reduceRegion() can be adjusted to reduce time or fine tune resolution
  def Reduce_median(img):
    median = img.reduceRegion(reducer=ee.Reducer.median(),geometry=studyArea,scale=500,bestEffort=True).get(band)
    variance = img.reduceRegion(reducer=ee.Reducer.variance(),geometry=studyArea,scale=500,bestEffort=True).get(band)
    return img.set('date',img.date().format()).set('Median', median).set('Variance',variance)

  # A for loop is used here to reduce the regions for every year of interest.
  # This is also a way to bypass Google Earth Engine stopping you because it is
  # too computationally heavy. The values for i_year and f_year are established 
  # via the widget in the cell above and are dynamically updated

  for i in range(i_year,f_year):

    # Access MODIS data with new date 
    collection = imageCollection
    MODIS = collection.filter(ee.Filter.date((str(i)+'-08-01'), (str(i+1))+'-08-01')).filterBounds(studyArea)
    snowCover = MODIS.select(band)

    # Apply Reducer
    studyAreaMedian = snowCover.map(Reduce_median)
    
    # Create/concatenate DataFrames 
    nested_list = studyAreaMedian.reduceColumns(ee.Reducer.toList(3),['date', 'Median','Variance']).values().get(0)
    if i == i_year: # If this is the first year, create a DataFrame
      df = pd.DataFrame(nested_list.getInfo(),columns=['date', 'Median','Variance'])
    else: # For every year that isn't the initial year, concatenate the new
          # DataFrames to the original DataFrame
      df2 = pd.DataFrame(nested_list.getInfo(),columns=['date', 'Median','Variance'])
      df=pd.concat([df,df2],ignore_index=True)

    # Add to the loading bar
    Reducer_Progress.value+=1
  
  df['date'] = pd.to_datetime(df['date'])

  return df


# Import shapefile and run 

In [6]:
# Import shapefile for NPR-A

NPRA_path = "/content/drive/MyDrive/Term/Working Code/Omeed/Shapefiles/NPR-A/study_area.shp"
NPRA_studyArea = studyAreaImporter(NPRA_path)

# Import shapefile for ANWR

ANWR_path = "/content/drive/MyDrive/Term/Working Code/Omeed/Shapefiles/ANWR/study_area.shp"
ANWR_studyArea = studyAreaImporter(ANWR_path)


# Map Variability in binary masks of Snow Cover for NPR-A

In [9]:
# Create geemap and our progress/loading bar
Map = geemap.Map()

# Select MODIS NDSI_Snow_Cover product and 
  # clip to studyArea
  
snowCover = ee.ImageCollection("MODIS/006/MOD10A1").filterBounds(NPRA_studyArea) \
.select('NDSI_Snow_Cover').map(lambda img : img.clip(NPRA_studyArea))



# Set up loading bar

i_year = year_range.value[0]
f_year = year_range.value[1]

Progress_Bar = widgets.IntProgress(
    value=0,
    min=0,
    max=(f_year-i_year)*16,
    step=1,
    description='Calculating Binary Changes',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal')

display(Progress_Bar)




# The .map function above allows us to iterate through all images in a collection and
  # apply our snowCoverThreshold function.

# Pre-allocate lists to store our images in according to the for-loop below:
JanuaryList = ee.List([])
FebruaryList = ee.List([])
MarchList = ee.List([])
AprilList = ee.List([])
MayList = ee.List([])
JuneList = ee.List([])
JulyList = ee.List([])
AugustList = ee.List([])
SeptemberList = ee.List([])
OctoberList = ee.List([])
NovemberList = ee.List([])
DecemberList = ee.List([])

# Filter the data by the bins of months/years of interest using a for loop
  # years will be the range between i_year and f_year + 1, and month bin size
  # will be 4 months (which divides the year by 3) starting in September (month = 9),
  # then January (month = 1), then May (month = 5)


for year in range(i_year, f_year):

  # Filter through our snowOnOff ImageCollection using "year" and ee.Filter.calendarRange()
  snowCover_yearFilter = snowCover.filter(ee.Filter.date((str(year)+'-09-01'),(str(year+1)+'-08-31')))
  for month in range(1,13):
    # Filter through the month bins of our snowOnOff_yearFilter using "month" and ee.Filter.calendarRange()
    snowCover_monthFilter = snowCover_yearFilter.filter(ee.Filter.calendarRange(month, month, 'month'))

    # Create binary masks where snowCover is >= 15%

    def snowCoverThreshold(img):
      return img.gte(15) 

    snowOnOff_monthFilter = snowCover_monthFilter.map(snowCoverThreshold)

    ''' 
--------------------------------------------------------------------------------
        Now we will begin iterating through the ImageCollections and detecting
        changes between images. Finally, we will add these changes into the
        corresponding monthly bins 
    '''

    # Convert our ImageCollection to an earth engine list
    snowOnOffList = snowOnOff_monthFilter.toList(snowOnOff_monthFilter.size())

    # Generate a list of indexes from 1st image to 2nd last image as
      # we are trying to get difference between two images

    indexes = snowOnOffList.size()
    indexes = ee.List.sequence(0,indexes.subtract(2))

    # Now apply a mapping function (.map) that takes an image at our target index and
      # the next index to check if the snow cover binaries are the same
    def check_change(index):
      index = ee.Number(index)
      thisImg = ee.Image(snowOnOffList.get(index))
      nextImg = ee.Image(snowOnOffList.get(index.add(1)))
      return thisImg.neq(nextImg)
    changeList = ee.List([])
    changeList = indexes.map(check_change)

    # Each pixel of each image in the list now has a value 0 or 1 value from the neq
      # operation which means each change has value 1. We will concatenate these lists
      # into their corresponding bins using if statements and the .cat function.


    if month == 1:
      JanuaryList = JanuaryList.cat(changeList)
    elif month == 2:
      FebruaryList = FebruaryList.cat(changeList)
    elif month == 3:
      MarchList = MarchList.cat(changeList)
    elif month == 4:
      AprilList = AprilList.cat(changeList)
    elif month == 5:
      MayList=MayList.cat(changeList)
    elif month == 6:
      JuneList=JuneList.cat(changeList)
    elif month == 7:
      JulyList=JulyList.cat(changeList)
    elif month == 8:
      AugustList=AugustList.cat(changeList)
    elif month == 9:
      SeptemberList=SeptemberList.cat(changeList)
    elif month == 10:
      OctoberList=OctoberList.cat(changeList)
    elif month == 11:
      NovemberList=NovemberList.cat(changeList)
    elif month == 12:
      DecemberList=DecemberList.cat(changeList)


      # Here we will add one value to the loading bar to give us an indication of
        # how long things will take. We will also change the description to let
        # us know what year we are at. 
      
    Progress_Bar.value += 1
    Progress_Bar.description = ("Calculating Binary Changes for " + str(year) + "to " + str(year+1))

'''

End of For-Loop
--------------------------------------------------------------------------------

'''

# Now we exit the for-loop and turn our list of images into an image of the sum 
  # of pixel changes. We will collect these images and their descriptors in lists
  # for use in a for-loop below. 

januaryImage = ee.ImageCollection.fromImages(JanuaryList).sum()
jan_descriptor = "January Snow Cover Variability"
februaryImage = ee.ImageCollection.fromImages(FebruaryList).sum()
feb_descriptor = "February Snow Cover Variability"
marchImage = ee.ImageCollection.fromImages(MarchList).sum()
march_descriptor = "March Snow Cover Variability"
aprilImage = ee.ImageCollection.fromImages(AprilList).sum()
april_descriptor = "April Snow Cover Variability"
mayImage = ee.ImageCollection.fromImages(MayList).sum()
may_descriptor = "May Snow Cover Variability"
juneImage = ee.ImageCollection.fromImages(JuneList).sum()
june_descriptor = "June Snow Cover Variability"
julyImage = ee.ImageCollection.fromImages(JulyList).sum()
july_descriptor = "July Snow Cover Variability"
augustImage = ee.ImageCollection.fromImages(AugustList).sum()
august_descriptor = "August Snow Cover Variability"
septemberImage = ee.ImageCollection.fromImages(SeptemberList).sum()
september_descriptor = "September Snow Cover Variability"
octoberImage = ee.ImageCollection.fromImages(OctoberList).sum()
october_descriptor = "October Snow Cover Variability"
novemberImage = ee.ImageCollection.fromImages(NovemberList).sum()
november_descriptor = "November Snow Cover Variability"
decemberImage = ee.ImageCollection.fromImages(DecemberList).sum()
december_descriptor = "December Snow Cover Variability"

# Collect the images and descriptors in the list
img_list = [januaryImage,februaryImage,marchImage,aprilImage,mayImage,juneImage,julyImage,augustImage,septemberImage,octoberImage,novemberImage,decemberImage]
descriptor_list = [jan_descriptor,feb_descriptor,march_descriptor,april_descriptor,may_descriptor,june_descriptor,july_descriptor,august_descriptor,september_descriptor,october_descriptor,november_descriptor,december_descriptor]

# Using the for-loop below, we will add our images to our snowOnOff_Map.
# Run the for-loop:

max_value = []

for i in range(len(img_list)):
  

  # Below we find the max-values of each image and define our vis_params:
  img = img_list[i]
  # max_value.append(img.reduceRegion(ee.Reducer.max(), geometry=studyArea, scale=500, bestEffort = True).getInfo()['NDSI_Snow_Cover'])
  vis_params = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    'min': 0,
    'max': 50
    # 'gain':1
    }

    # We now add the images to our "snowOnOff_Map" with the vis_params.

  descriptor = descriptor_list[i]
  Map.addLayer(img, vis_params, descriptor)
  Progress_Bar.value += 1


# Filter our max_values for None values
# filtered_max = filter(lambda item: item is not None, max_value)
# filtered_max = list(filtered_max)
# Add a layer control option and a color scale to the map
Map.addLayerControl()

color_scale = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    'min':0,
    'max':50
    # 'gain':1
    }

Map.add_colorbar(color_scale)

# Display the map

Map

IntProgress(value=0, bar_style='info', description='Calculating Binary Changes', max=352)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Make Gif of Snow On/Off Variability

In [10]:
# Make an ee.List of the images we made
gifList = ee.List(img_list)
# Turn the ee.List into an ImageCollection
gifCollection = ee.ImageCollection.fromImages(gifList)

# Define a mask to clip the NDSI data by.
mask = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Alaska'))

# Define visualization parameters.
visParams = {
#  'min': 0.0,
#  'max': max(filtered_max),
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}

color_scale = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    # 'gain':1
    'min':0,
    'max':50
    }

# Create RGB visualization images for use as animation frames.
gifCollectionClipped = gifCollection.map(lambda img: img.visualize(**color_scale).clip(mask))
region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)

gifParams = {
  # 'region': studyArea.geometry(),
  'region' : region,
  'dimensions': 600,
  'crs': 'EPSG:3338', # Updated the crs to Alaska's EPSG
  'framesPerSecond': 3,
  'format': 'gif'
}

# Print the GIF URL to the console.
url_NPRA = gifCollectionClipped.getVideoThumbURL(gifParams)

# Map Variability in binary masks of Snow Cover for ANWR

In [13]:
# Create geemap and our progress/loading bar
Map = geemap.Map()

# Select MODIS NDSI_Snow_Cover product and 
  # clip to studyArea
  
snowCover = ee.ImageCollection("MODIS/006/MOD10A1").filterBounds(ANWR_studyArea) \
.select('NDSI_Snow_Cover').map(lambda img : img.clip(ANWR_studyArea))



# Set up loading bar

i_year = year_range.value[0]
f_year = year_range.value[1]

Progress_Bar = widgets.IntProgress(
    value=0,
    min=0,
    max=(f_year-i_year)*16,
    step=1,
    description='Calculating Binary Changes',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal')

display(Progress_Bar)




# The .map function above allows us to iterate through all images in a collection and
  # apply our snowCoverThreshold function.

# Pre-allocate lists to store our images in according to the for-loop below:
JanuaryList = ee.List([])
FebruaryList = ee.List([])
MarchList = ee.List([])
AprilList = ee.List([])
MayList = ee.List([])
JuneList = ee.List([])
JulyList = ee.List([])
AugustList = ee.List([])
SeptemberList = ee.List([])
OctoberList = ee.List([])
NovemberList = ee.List([])
DecemberList = ee.List([])

# Filter the data by the bins of months/years of interest using a for loop
  # years will be the range between i_year and f_year + 1, and month bin size
  # will be 4 months (which divides the year by 3) starting in September (month = 9),
  # then January (month = 1), then May (month = 5)


for year in range(i_year, f_year):

  # Filter through our snowOnOff ImageCollection using "year" and ee.Filter.calendarRange()
  snowCover_yearFilter = snowCover.filter(ee.Filter.date((str(year)+'-09-01'),(str(year+1)+'-08-31')))
  for month in range(1,13):
    # Filter through the month bins of our snowOnOff_yearFilter using "month" and ee.Filter.calendarRange()
    snowCover_monthFilter = snowCover_yearFilter.filter(ee.Filter.calendarRange(month, month, 'month'))

    # Create binary masks where snowCover is >= 15%

    def snowCoverThreshold(img):
      return img.gte(15) 

    snowOnOff_monthFilter = snowCover_monthFilter.map(snowCoverThreshold)

    ''' 
--------------------------------------------------------------------------------
        Now we will begin iterating through the ImageCollections and detecting
        changes between images. Finally, we will add these changes into the
        corresponding monthly bins 
    '''

    # Convert our ImageCollection to an earth engine list
    snowOnOffList = snowOnOff_monthFilter.toList(snowOnOff_monthFilter.size())

    # Generate a list of indexes from 1st image to 2nd last image as
      # we are trying to get difference between two images

    indexes = snowOnOffList.size()
    indexes = ee.List.sequence(0,indexes.subtract(2))

    # Now apply a mapping function (.map) that takes an image at our target index and
      # the next index to check if the snow cover binaries are the same
    def check_change(index):
      index = ee.Number(index)
      thisImg = ee.Image(snowOnOffList.get(index))
      nextImg = ee.Image(snowOnOffList.get(index.add(1)))
      return thisImg.neq(nextImg)
    changeList = ee.List([])
    changeList = indexes.map(check_change)

    # Each pixel of each image in the list now has a value 0 or 1 value from the neq
      # operation which means each change has value 1. We will concatenate these lists
      # into their corresponding bins using if statements and the .cat function.


    if month == 1:
      JanuaryList = JanuaryList.cat(changeList)
    elif month == 2:
      FebruaryList = FebruaryList.cat(changeList)
    elif month == 3:
      MarchList = MarchList.cat(changeList)
    elif month == 4:
      AprilList = AprilList.cat(changeList)
    elif month == 5:
      MayList=MayList.cat(changeList)
    elif month == 6:
      JuneList=JuneList.cat(changeList)
    elif month == 7:
      JulyList=JulyList.cat(changeList)
    elif month == 8:
      AugustList=AugustList.cat(changeList)
    elif month == 9:
      SeptemberList=SeptemberList.cat(changeList)
    elif month == 10:
      OctoberList=OctoberList.cat(changeList)
    elif month == 11:
      NovemberList=NovemberList.cat(changeList)
    elif month == 12:
      DecemberList=DecemberList.cat(changeList)


      # Here we will add one value to the loading bar to give us an indication of
        # how long things will take. We will also change the description to let
        # us know what year we are at. 
      
    Progress_Bar.value += 1
    Progress_Bar.description = ("Calculating Binary Changes for " + str(year) + "to " + str(year+1))

'''

End of For-Loop
--------------------------------------------------------------------------------

'''

# Now we exit the for-loop and turn our list of images into an image of the sum 
  # of pixel changes. We will collect these images and their descriptors in lists
  # for use in a for-loop below. 

januaryImage = ee.ImageCollection.fromImages(JanuaryList).sum()
jan_descriptor = "January Snow Cover Variability"
februaryImage = ee.ImageCollection.fromImages(FebruaryList).sum()
feb_descriptor = "February Snow Cover Variability"
marchImage = ee.ImageCollection.fromImages(MarchList).sum()
march_descriptor = "March Snow Cover Variability"
aprilImage = ee.ImageCollection.fromImages(AprilList).sum()
april_descriptor = "April Snow Cover Variability"
mayImage = ee.ImageCollection.fromImages(MayList).sum()
may_descriptor = "May Snow Cover Variability"
juneImage = ee.ImageCollection.fromImages(JuneList).sum()
june_descriptor = "June Snow Cover Variability"
julyImage = ee.ImageCollection.fromImages(JulyList).sum()
july_descriptor = "July Snow Cover Variability"
augustImage = ee.ImageCollection.fromImages(AugustList).sum()
august_descriptor = "August Snow Cover Variability"
septemberImage = ee.ImageCollection.fromImages(SeptemberList).sum()
september_descriptor = "September Snow Cover Variability"
octoberImage = ee.ImageCollection.fromImages(OctoberList).sum()
october_descriptor = "October Snow Cover Variability"
novemberImage = ee.ImageCollection.fromImages(NovemberList).sum()
november_descriptor = "November Snow Cover Variability"
decemberImage = ee.ImageCollection.fromImages(DecemberList).sum()
december_descriptor = "December Snow Cover Variability"

# Collect the images and descriptors in the list
img_list = [januaryImage,februaryImage,marchImage,aprilImage,mayImage,juneImage,julyImage,augustImage,septemberImage,octoberImage,novemberImage,decemberImage]
descriptor_list = [jan_descriptor,feb_descriptor,march_descriptor,april_descriptor,may_descriptor,june_descriptor,july_descriptor,august_descriptor,september_descriptor,october_descriptor,november_descriptor,december_descriptor]

# Using the for-loop below, we will add our images to our snowOnOff_Map.
# Run the for-loop:

max_value = []

for i in range(len(img_list)):
  

  # Below we find the max-values of each image and define our vis_params:
  img = img_list[i]
  # max_value.append(img.reduceRegion(ee.Reducer.max(), geometry=studyArea, scale=500, bestEffort = True).getInfo()['NDSI_Snow_Cover'])
  vis_params = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    'min': 0,
    'max': 50
    # 'gain':1
    }

    # We now add the images to our "snowOnOff_Map" with the vis_params.

  descriptor = descriptor_list[i]
  Map.addLayer(img, vis_params, descriptor)
  Progress_Bar.value += 1


# Filter our max_values for None values
# filtered_max = filter(lambda item: item is not None, max_value)
# filtered_max = list(filtered_max)
# Add a layer control option and a color scale to the map
Map.addLayerControl()

color_scale = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    'min':0,
    'max':50,
    # 'gain':1
    }

Map.add_colorbar(color_scale)

# Display the map

Map

IntProgress(value=0, bar_style='info', description='Calculating Binary Changes', max=352)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Make Gif of Snow On/Off Variability

In [14]:
# Make an ee.List of the images we made
gifList = ee.List(img_list)
# Turn the ee.List into an ImageCollection
gifCollection = ee.ImageCollection.fromImages(gifList)

# Define a mask to clip the NDSI data by.
mask = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Alaska'))

# Define visualization parameters.
color_scale = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921'],
    'min':0,
    'max':50
    }

# Create RGB visualization images for use as animation frames.
gifCollectionClipped = gifCollection.map(lambda img: img.visualize(**color_scale).clip(mask))

region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)

gifParams = {
  # 'region': studyArea.geometry(),
  'region' : region,
  'dimensions': 600,
  'crs': 'EPSG:3338', # Updated the crs to Alaska's EPSG
  'framesPerSecond': 3,
  'format': 'gif'
}

# Print the GIF URL to the console.
url_ANWR = gifCollectionClipped.getVideoThumbURL(gifParams)


# Print Gifs of Both Regions

In [15]:
print("NPRA:"+url_NPRA)
print("ANWR:"+url_ANWR)

NPRA:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0dffce7d6cd3e7d4e5c9546e4ca1d8f5-afe929ecf894025e0892f940a2f9cb89:getPixels
ANWR:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/ae8c6007580a8ef6bd4b01e9298ef0d2-fd92ed84f43e3a8e3c82ca132a58b0d6:getPixels


In [16]:
# Download animation to Google Drive.
import urllib.request
gif_name_ANWR = 'ANWR.gif'
urllib.request.urlretrieve(url_ANWR, gif_name_ANWR)

('ANWR.gif', <http.client.HTTPMessage at 0x7ff291a9bc40>)

In [17]:
gif_name_NPRA = 'NPR_A.gif'
urllib.request.urlretrieve(url_NPRA,gif_name_NPRA)

('NPR_A.gif', <http.client.HTTPMessage at 0x7ff291a9b400>)

# Identifying Annual First/Last Day of Snow

In [18]:
# Identifying Annual First Day of Snow for NPR-A
startDoy = 1
startYear = 2000
endYear = 2022
startDate = None

def addDateBands(img):
    # Get image date.
    date = img.date()
    # Get calendar day-of-year.
    calDoy = date.getRelative('day', 'year')
    # Get relative day-of-year; enumerate from user-defined startDoy.
    relDoy = date.difference(startDate, 'day')
    # Get the date as milliseconds from Unix epoch.
    millis = date.millis()
    # Add all of the above date info as bands to the snow fraction image.
    dateBands = ee.Image.constant([calDoy, relDoy, millis, startYear]).rename(['calDoy', 'relDoy', 'millis', 'year'])
    # Cast bands to correct data type before returning the image.
    return img.addBands(dateBands).cast({'calDoy': 'int', 'relDoy': 'int', 'millis': 'long', 'year': 'int'}).set('millis', millis)

waterMask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').Not()

completeCol = ee.ImageCollection('MODIS/006/MOD10A1').filterBounds(NPRA_studyArea.geometry()).select('NDSI_Snow_Cover').map(lambda img : img.clip(NPRA_studyArea))

# Pixels must have been 10% snow covered for at least 2 weeks in 2018.
snowCoverEphem = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().gte(14)

# Pixels must not be 10% snow covered more than 124 days in 2018.
snowCoverConst = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().lte(124)

analysisMask = waterMask.multiply(snowCoverEphem).multiply(snowCoverConst)

years = ee.List.sequence(startYear, endYear)

def annualList(year):
    global startYear, startDate
    # Set the global startYear variable as the year being worked on so that
    # it will be accessible to the addDateBands mapped to the collection below.
    startYear = year

    # Get the first day-of-year for this year as an ee.Date object.
    firstDoy = ee.Date.fromYMD(year, 1, 1)

    # Advance from the firstDoy to the user-defined startDay; subtract 1 since
    # firstDoy is already 1. Set the result as the global startDate variable so
    # that it is accessible to the addDateBands mapped to the collection below.
    startDate = firstDoy.advance(startDoy - 1, 'day')

    # Get endDate for this year by advancing 1 year from startDate.
    # Need to advance an extra day because end date of filterDate() function
    # is exclusive.
    endDate = startDate.advance(1, 'year').advance(1, 'day')

    # Filter the complete collection by the start and end dates just defined.
    yearCol = completeCol.filterDate(startDate, endDate)

    # Construct an image where pixels represent the first day within the date
    # range that the lowest snow fraction is observed.
    noSnowImg = yearCol \
        .map(addDateBands) \
        .sort('millis',False) \
        .reduce(ee.Reducer.min(5)) \
        .rename(['snowCover', 'calDoy', 'relDoy', 'millis', 'year']) \
        .updateMask(analysisMask) \
        .set('year',year)
    return noSnowImg.updateMask(noSnowImg.select('snowCover').eq(0))

# Collect the images the images by parsing through each year of our study period
annualCol = ee.ImageCollection.fromImages(years.map(annualList))
imgList = ee.List([])
for year in range(2000,2023):
  thisYear = year
  palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921']

  visArgs = {
    'bands': ['calDoy'],
    'min': 230,
    'max': 290,
    'palette': palette}

  firstDaySnowYear = annualCol.filter(ee.Filter.eq('year',thisYear)).first()

  imgList = imgList.add(firstDaySnowYear)
firstDaySnowNPRA = ee.ImageCollection.fromImages(imgList)

In [19]:
# Identifying Annual First Day of Snow for ANWR
startDoy = 1
startYear = 2000
endYear = 2022
startDate = None

def addDateBands(img):
    # Get image date.
    date = img.date()
    # Get calendar day-of-year.
    calDoy = date.getRelative('day', 'year')
    # Get relative day-of-year; enumerate from user-defined startDoy.
    relDoy = date.difference(startDate, 'day')
    # Get the date as milliseconds from Unix epoch.
    millis = date.millis()
    # Add all of the above date info as bands to the snow fraction image.
    dateBands = ee.Image.constant([calDoy, relDoy, millis, startYear]).rename(['calDoy', 'relDoy', 'millis', 'year'])
    # Cast bands to correct data type before returning the image.
    return img.addBands(dateBands).cast({'calDoy': 'int', 'relDoy': 'int', 'millis': 'long', 'year': 'int'}).set('millis', millis)

waterMask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').Not()

completeCol = ee.ImageCollection('MODIS/006/MOD10A1').filterBounds(ANWR_studyArea.geometry()).select('NDSI_Snow_Cover').map(lambda img : img.clip(ANWR_studyArea))

# Pixels must have been 10% snow covered for at least 2 weeks in 2018.
snowCoverEphem = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().gte(14)

# Pixels must not be 10% snow covered more than 124 days in 2018.
snowCoverConst = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().lte(124)

analysisMask = waterMask.multiply(snowCoverEphem).multiply(snowCoverConst)

years = ee.List.sequence(startYear, endYear)

def annualList(year):
    global startYear, startDate
    # Set the global startYear variable as the year being worked on so that
    # it will be accessible to the addDateBands mapped to the collection below.
    startYear = year

    # Get the first day-of-year for this year as an ee.Date object.
    firstDoy = ee.Date.fromYMD(year, 1, 1)

    # Advance from the firstDoy to the user-defined startDay; subtract 1 since
    # firstDoy is already 1. Set the result as the global startDate variable so
    # that it is accessible to the addDateBands mapped to the collection below.
    startDate = firstDoy.advance(startDoy - 1, 'day')

    # Get endDate for this year by advancing 1 year from startDate.
    # Need to advance an extra day because end date of filterDate() function
    # is exclusive.
    endDate = startDate.advance(1, 'year').advance(1, 'day')

    # Filter the complete collection by the start and end dates just defined.
    yearCol = completeCol.filterDate(startDate, endDate)

    # Construct an image where pixels represent the first day within the date
    # range that the lowest snow fraction is observed.
    noSnowImg = yearCol \
        .map(addDateBands) \
        .sort('millis',False) \
        .reduce(ee.Reducer.min(5)) \
        .rename(['snowCover', 'calDoy', 'relDoy', 'millis', 'year']) \
        .updateMask(analysisMask) \
        .set('year',year)
    return noSnowImg.updateMask(noSnowImg.select('snowCover').eq(0))

# Collect the images the images by parsing through each year of our study period
annualCol = ee.ImageCollection.fromImages(years.map(annualList))
imgList = ee.List([])
for year in range(2000,2023):
  thisYear = year
  palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921']
  visArgs = {
    'bands': ['calDoy'],
    'min': 230,
    'max': 290,
    'palette': palette}

  firstDaySnowYear = annualCol.filter(ee.Filter.eq('year',thisYear)).first()
  imgList = imgList.add(firstDaySnowYear)
firstDaySnowANWR = ee.ImageCollection.fromImages(imgList)


In [20]:
# Identifying Annual Last Day of Snow for NPR-A
startDoy = 1
startYear = 2000
endYear = 2022
startDate = None

def addDateBands(img):
    # Get image date.
    date = img.date()
    # Get calendar day-of-year.
    calDoy = date.getRelative('day', 'year')
    # Get relative day-of-year; enumerate from user-defined startDoy.
    relDoy = date.difference(startDate, 'day')
    # Get the date as milliseconds from Unix epoch.
    millis = date.millis()
    # Add all of the above date info as bands to the snow fraction image.
    dateBands = ee.Image.constant([calDoy, relDoy, millis, startYear]).rename(['calDoy', 'relDoy', 'millis', 'year'])
    # Cast bands to correct data type before returning the image.
    return img.addBands(dateBands).cast({'calDoy': 'int', 'relDoy': 'int', 'millis': 'long', 'year': 'int'}).set('millis', millis)

waterMask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').Not()

completeCol = ee.ImageCollection('MODIS/006/MOD10A1').filterBounds(NPRA_studyArea.geometry()).select('NDSI_Snow_Cover').map(lambda img : img.clip(NPRA_studyArea))

# Pixels must have been 10% snow covered for at least 2 weeks in 2018.
snowCoverEphem = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().gte(14)

# Pixels must not be 10% snow covered more than 124 days in 2018.
snowCoverConst = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().lte(124)

analysisMask = waterMask.multiply(snowCoverEphem).multiply(snowCoverConst)

years = ee.List.sequence(startYear, endYear)

def annualList(year):
    global startYear, startDate
    # Set the global startYear variable as the year being worked on so that
    # it will be accessible to the addDateBands mapped to the collection below.
    startYear = year

    # Get the first day-of-year for this year as an ee.Date object.
    firstDoy = ee.Date.fromYMD(year, 1, 1)

    # Advance from the firstDoy to the user-defined startDay; subtract 1 since
    # firstDoy is already 1. Set the result as the global startDate variable so
    # that it is accessible to the addDateBands mapped to the collection below.
    startDate = firstDoy.advance(startDoy - 1, 'day')

    # Get endDate for this year by advancing 1 year from startDate.
    # Need to advance an extra day because end date of filterDate() function
    # is exclusive.
    endDate = startDate.advance(1, 'year').advance(1, 'day')

    # Filter the complete collection by the start and end dates just defined.
    yearCol = completeCol.filterDate(startDate, endDate)

    # Construct an image where pixels represent the first day within the date
    # range that the lowest snow fraction is observed.
    noSnowImg = yearCol \
        .map(addDateBands) \
        .sort('millis') \
        .reduce(ee.Reducer.min(5)) \
        .rename(['snowCover', 'calDoy', 'relDoy', 'millis', 'year']) \
        .updateMask(analysisMask) \
        .set('year',year)
    return noSnowImg.updateMask(noSnowImg.select('snowCover').eq(0))

# Collect the images the images by parsing through each year of our study period
annualCol = ee.ImageCollection.fromImages(years.map(annualList))
imgList = ee.List([])
for year in range(2000,2023):
  thisYear = year
  palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921']

  visArgs = {
    'bands': ['calDoy'],
    'min': 150,
    'max': 200,
    'palette': palette}

  lastDaySnowYear = annualCol.filter(ee.Filter.eq('year',thisYear)).first()

  imgList = imgList.add(lastDaySnowYear)
lastDaySnowNPRA = ee.ImageCollection.fromImages(imgList)


In [21]:
# Identifying Annual Last Day of Snow for NPR-A
startDoy = 1
startYear = 2000
endYear = 2022
startDate = None

def addDateBands(img):
    # Get image date.
    date = img.date()
    # Get calendar day-of-year.
    calDoy = date.getRelative('day', 'year')
    # Get relative day-of-year; enumerate from user-defined startDoy.
    relDoy = date.difference(startDate, 'day')
    # Get the date as milliseconds from Unix epoch.
    millis = date.millis()
    # Add all of the above date info as bands to the snow fraction image.
    dateBands = ee.Image.constant([calDoy, relDoy, millis, startYear]).rename(['calDoy', 'relDoy', 'millis', 'year'])
    # Cast bands to correct data type before returning the image.
    return img.addBands(dateBands).cast({'calDoy': 'int', 'relDoy': 'int', 'millis': 'long', 'year': 'int'}).set('millis', millis)

waterMask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').Not()

completeCol = ee.ImageCollection('MODIS/006/MOD10A1').filterBounds(ANWR_studyArea.geometry()).select('NDSI_Snow_Cover').map(lambda img : img.clip(ANWR_studyArea))

# Pixels must have been 10% snow covered for at least 2 weeks in 2018.
snowCoverEphem = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().gte(14)

# Pixels must not be 10% snow covered more than 124 days in 2018.
snowCoverConst = completeCol.filterDate('2018-01-01', '2019-01-01').map(lambda img: img.gte(15)).sum().lte(124)

analysisMask = waterMask.multiply(snowCoverEphem).multiply(snowCoverConst)

years = ee.List.sequence(startYear, endYear)

def annualList(year):
    global startYear, startDate
    # Set the global startYear variable as the year being worked on so that
    # it will be accessible to the addDateBands mapped to the collection below.
    startYear = year

    # Get the first day-of-year for this year as an ee.Date object.
    firstDoy = ee.Date.fromYMD(year, 1, 1)

    # Advance from the firstDoy to the user-defined startDay; subtract 1 since
    # firstDoy is already 1. Set the result as the global startDate variable so
    # that it is accessible to the addDateBands mapped to the collection below.
    startDate = firstDoy.advance(startDoy - 1, 'day')

    # Get endDate for this year by advancing 1 year from startDate.
    # Need to advance an extra day because end date of filterDate() function
    # is exclusive.
    endDate = startDate.advance(1, 'year').advance(1, 'day')

    # Filter the complete collection by the start and end dates just defined.
    yearCol = completeCol.filterDate(startDate, endDate)

    # Construct an image where pixels represent the first day within the date
    # range that the lowest snow fraction is observed.
    noSnowImg = yearCol \
        .map(addDateBands) \
        .sort('millis') \
        .reduce(ee.Reducer.min(5)) \
        .rename(['snowCover', 'calDoy', 'relDoy', 'millis', 'year']) \
        .updateMask(analysisMask) \
        .set('year',year)
    return noSnowImg.updateMask(noSnowImg.select('snowCover').eq(0))

# Collect the images the images by parsing through each year of our study period
annualCol = ee.ImageCollection.fromImages(years.map(annualList))
imgList = ee.List([])
for year in range(2000,2023):
  thisYear = year
  palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921']
  reversed_palette = palette[::-1]
  visArgs = {
    'bands': ['calDoy'],
    'min': 150,
    'max': 200,
    'palette': palette}

  lastDaySnowYear = annualCol.filter(ee.Filter.eq('year',thisYear)).first()

  imgList = imgList.add(lastDaySnowYear)
lastDaySnowANWR = ee.ImageCollection.fromImages(imgList)

In [22]:
# Define a mask to clip the data by.
# mask = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Alaska'))

FSD_visParams = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F'],
    'min':230,
    'max':290,
    'bands': ['calDoy']
}

LSD_visParams = {
    'palette':['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F'],
    'min':150,
    'max':200,
    'bands': ['calDoy']
}

# Set a region (box) around ANWR
ANWR_region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)
# Set a region (box) around NPRA
NPRA_region = ee.Geometry.Polygon(
    [[[-149.90414749199374,67.92726660651483],
      [-149.90414749199374,71.81689549403126],
      [-163.43930374199374,71.81689549403126],
      [-163.43930374199374,67.92726660651483]]],
    None, False
)

# Create visualization images for use as animation frames.
FSD_ANWRgifCollection = firstDaySnowANWR.map(lambda img: img.visualize(**FSD_visParams))
FSD_NPRAgifCollection = firstDaySnowNPRA.map(lambda img: img.visualize(**FSD_visParams))
LSD_ANWRgifCollection = lastDaySnowANWR.map(lambda img: img.visualize(**LSD_visParams))
LSD_NPRAgifCollection = lastDaySnowNPRA.map(lambda img: img.visualize(**LSD_visParams))

# Construct our gifParameters
ANWRgifParams = {
  'region' : ANWR_region,
  'dimensions': 768,
  'crs': 'EPSG:3338', # The crs for Alaska
  'framesPerSecond': 2,
  'format': 'gif'
}

NPRAgifParams = {
  'region' : NPRA_region,
  'dimensions': 768,
  'crs': 'EPSG:3338', # The crs for Alaska
  'framesPerSecond': 2,
  'format': 'gif'
}

# Save the GIF URL to the following variables:
url_FSDANWR = FSD_ANWRgifCollection.getVideoThumbURL(ANWRgifParams)
url_FSDNPRA = FSD_NPRAgifCollection.getVideoThumbURL(NPRAgifParams)
url_LSDANWR = LSD_ANWRgifCollection.getVideoThumbURL(ANWRgifParams)
url_LSDNPRA = LSD_NPRAgifCollection.getVideoThumbURL(NPRAgifParams)

# Print the URLs
print("ANWR First Day of Snow:"+url_FSDANWR)
print("NPRA First Day of Snow:"+url_FSDNPRA)
print("ANWR Last Day of Snow:"+url_LSDANWR)
print("NPRA Last Day of Snow:"+url_LSDNPRA)


ANWR First Day of Snow:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/e2d107b5658b1ef0a9d7017705ac65e1-e24333257702001978ff09c80bdcaf40:getPixels
NPRA First Day of Snow:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/180f53b0428410df2ee211f65132f739-1a5d68cdca17ea0032fb41492ab2ebd4:getPixels
ANWR Last Day of Snow:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/8692fb0468fa85d4458f4508ad4b9af2-6100da30515c79998e84859ba859fa1e:getPixels
NPRA Last Day of Snow:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/06f9d2c8dde173e4a4fb823799916835-3a8f265c2a4630557fcf8e1ebf4a2801:getPixels


In [ ]:
# Download animation to Google Drive.
import urllib.request
urllib.request.urlretrieve(url_FSDANWR, 'FirstDaySnow_ANWR_min:230_max:290.gif')
urllib.request.urlretrieve(url_FSDNPRA, 'FirstDaySnow_NPRA_min:230_max:290.gif')
urllib.request.urlretrieve(url_LSDANWR, 'LastDaySnow_ANWR_min:140_max:200.gif')
urllib.request.urlretrieve(url_LSDNPRA, 'LastDaySnow_NPRA_min:140_max:200.gif')

# Calculating Snow Season Length

In [23]:
# Convert our ImageCollections into earth engine lists

# ANWR Collections
firstDaySnowANWR_list = firstDaySnowANWR.select('calDoy').toList(firstDaySnowANWR.size())
lastDaySnowANWR_list = lastDaySnowANWR.select('calDoy').toList(lastDaySnowANWR.size())
# NPRA Collections
firstDaySnowNPRA_list = firstDaySnowNPRA.select('calDoy').toList(firstDaySnowNPRA.size())
lastDaySnowNPRA_list = lastDaySnowNPRA.select('calDoy').toList(lastDaySnowNPRA.size())

# Generate a list of indexes from 1st image to 2nd last image as
# we are trying to get difference between two images

indexes = firstDaySnowANWR_list.size()

indexes = ee.List.sequence(0,indexes.subtract(2))


# Now apply a mapping function (.map) that takes an image at our target index and
# the next index to subtract the calDoy of each region. We will add these differences into lists.
def subtract_maps_ANWR(index):
  index = ee.Number(index)
  FSDImg = ee.Image(firstDaySnowANWR_list.get(index))
  LSDImg = ee.Image(lastDaySnowANWR_list.get(index.add(1)))
  return LSDImg.subtract(FSDImg).add(365)

def subtract_maps_NPRA(index):
  index = ee.Number(index)
  FSDImg = ee.Image(firstDaySnowNPRA_list.get(index))
  LSDImg = ee.Image(lastDaySnowNPRA_list.get(index.add(1)))
  return LSDImg.subtract(FSDImg).add(365)

# Apply the function
diffListANWR = ee.List([])
diffListANWR = indexes.map(subtract_maps_ANWR)

diffListNPRA = ee.List([])
diffListNPRA = indexes.map(subtract_maps_NPRA)

# Turn these list of images into Image Collections
snowSeasonLength_ANWR = ee.ImageCollection.fromImages(diffListANWR)
snowSeasonLength_NPRA = ee.ImageCollection.fromImages(diffListNPRA)


In [24]:
# Create visual parameter for the gifs
palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F', 'F0F921']

visArgs = {
  'bands': ['calDoy'],
  'min': 215,
  'max': 335,
  'palette': palette}

# Set a region (box) around ANWR
ANWR_region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)
# Set a region (box) around NPRA
NPRA_region = ee.Geometry.Polygon(
    [[[-149.90414749199374,67.92726660651483],
      [-149.90414749199374,71.81689549403126],
      [-163.43930374199374,71.81689549403126],
      [-163.43930374199374,67.92726660651483]]],
    None, False
)

# Create visualization images for use as animation frames.
snowSeasonLength_ANWR_gifCollection = snowSeasonLength_ANWR.map(lambda img: img.visualize(**visArgs))
snowSeasonLength_NPRA_gifCollection = snowSeasonLength_NPRA.map(lambda img: img.visualize(**visArgs))


# Construct our gifParameters
ANWRgifParams = {
  'region' : ANWR_region,
  'dimensions': 768,
  'crs': 'EPSG:3338', # The crs for Alaska
  'framesPerSecond': 2,
  'format': 'gif'
}

NPRAgifParams = {
  'region' : NPRA_region,
  'dimensions': 768,
  'crs': 'EPSG:3338', # The crs for Alaska
  'framesPerSecond': 2,
  'format': 'gif'
}

# Save the GIF URL to the following variables:
url_SSLANWR = snowSeasonLength_ANWR_gifCollection.getVideoThumbURL(ANWRgifParams)
url_SSLNPRA = snowSeasonLength_NPRA_gifCollection.getVideoThumbURL(NPRAgifParams)

# Print the URLs
print("ANWR Snow Season Length:"+url_SSLANWR)
print("NPRA Snow Season Length:"+url_SSLNPRA)


ANWR Snow Season Length:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/b2bbcbe02a153d5917111e5c796cbc1f-1d652845d9a245930bb9f74e74197af0:getPixels
NPRA Snow Season Length:https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/81992e300f2dc46e4d8c3e89fed58b4b-aebdb1efae0e53aa59941614f15977a5:getPixels


In [25]:
# Download animation to Google Drive.
import urllib.request
urllib.request.urlretrieve(url_SSLANWR, 'ANWR_Snow_Season_Length_min:215_max:335.gif')
urllib.request.urlretrieve(url_SSLNPRA, 'NPRA_Snow_Season_Length_min:215_max:335.gif')

('NPRA_Snow_Season_Length_min:215_max:335.gif',
 <http.client.HTTPMessage at 0x7ff28dbbf8b0>)